In [1]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pandas as pd
import numpy as np
import navis
from fafbseg import flywire
from compcon.navis_api import get_flywire_neuron, get_hemibrain_neuron
from compcon.create_graph import get_neuron, get_graph, draw_graph, get_neuron_local, overall, draw_connection, draw3d_graph
from compcon.isomporphic_subgraphs import find_isomorphic_subgraphs
import itertools
from skelescope.skelescope import Skelescope
from vimosketches.src.anywidget_test import Widget
import random

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.04.19


In [3]:
# Load the CSV file into a DataFrame
df = pd.read_csv('column_assignment.csv')

# Extract root_id and column_id
df = df[['root_id', 'column_id']]
ids=[]
for i in range(2):
    ids.extend(df[df["column_id"] == (i+1)]["root_id"].values.tolist())

In [4]:
# Load the CSV file
classification = pd.read_csv('classification.csv')

# Filter for 'optic' in 'super_class' and 'right' in 'side'
filtered_classification = classification[(classification['super_class'] == 'optic') & (classification['side'] == 'right')]

# Display the filtered DataFrame
# print(filtered_df)

# Get the 'root_id' for the filtered rows
root_ids = filtered_classification['root_id'].tolist()

# print("Filtered root IDs:")
# print(root_ids)


In [5]:
syn_co = pd.read_csv('synapse_coordinates.csv', dtype={'pre_root_id': str, 'post_root_id': str})

# Forward fill the empty values
syn_co['pre_root_id'].ffill(inplace=True)
syn_co['post_root_id'].ffill(inplace=True)

# Convert the columns to int64 now that the missing values are filled
syn_co['pre_root_id'] = syn_co['pre_root_id'].astype('int64')
syn_co['post_root_id'] = syn_co['post_root_id'].astype('int64')
# filtered_df = syn_co[syn_co['pre_root_id'].isin(root_ids) & syn_co['post_root_id'].isin(root_ids)]
# combos = filtered_df[['pre_root_id', 'post_root_id']].drop_duplicates().values.tolist()

In [6]:
# Assuming root_ids is your list of IDs
filtered_df = syn_co[syn_co['pre_root_id'].isin(root_ids) & syn_co['post_root_id'].isin(root_ids)]

In [7]:
combos = filtered_df[['pre_root_id', 'post_root_id']].drop_duplicates().values.tolist()

In [8]:
def snap_coordinates(src, dst, s_x, s_y, s_z, s_x_2, s_y_2, s_z_2, s_x_3, s_y_3, s_z_3, d_x, d_y, d_z, d_x_2, d_y_2, d_z_2, d_x_3, d_y_3, d_z_3, c_coordinates):
    src_2 = np.array(src) + 1000
    src_2 = list(src_2)

    src_3 = np.array(src) + 1000000
    src_3 = list(src_3)

    dst_2 = np.array(dst) + 1000
    dst_2 = list(dst_2)

    dst_3 = np.array(dst) + 1000000
    dst_3 = list(dst_3)

    syn_mapping = dict(zip(src + src_2 + src_3 + dst + dst_2 + dst_3, src + src + src + dst + dst + dst))
    
    data = {
        'x': s_x + s_x_2 + s_x_3 + d_x + d_x_2 + d_x_3,
        'y': s_y + s_y_2 + s_y_3 + d_y + d_y_2 + d_y_3,
        'z': s_z + s_z_2 + s_z_3 + d_z + d_z_2 + d_z_3,
        'node_id': src + src_2 + src_3 + dst + dst_2 + dst_3,
        'parent_id': [0] * len(src + src_2 + src_3 + dst + dst_2 + dst_3)
    }
    
    df = pd.DataFrame(data).copy()

    new_n = navis.TreeNeuron(
        df[['node_id', 'parent_id', 'x', 'y', 'z']],
        None
    )

    n_ids, dists = new_n.snap(c_coordinates)
    n_ids = list(n_ids)

    syn_ids = np.vectorize(syn_mapping.get)(n_ids)

    return syn_ids, dists



In [9]:
def synaptic_data(neurons, id_list):
    synaptic_src, synaptic_s_bef, synaptic_s_af, synaptic_d_bef, synaptic_d_af, synaptic_dst, x, y, z, n_combo, dists = [], [], [], [], [], [], [], [], [], [], []
    # # combinations = list(itertools.combinations(id_list, 2))
    # syn_co = pd.read_csv('synapse_coordinates.csv', dtype={'pre_root_id': str, 'post_root_id': str})

    # # Forward fill the empty values
    # syn_co['pre_root_id'].ffill(inplace=True)
    # syn_co['post_root_id'].ffill(inplace=True)
    
    # # Convert the columns to int64 now that the missing values are filled
    # syn_co['pre_root_id'] = syn_co['pre_root_id'].astype('int64')
    # syn_co['post_root_id'] = syn_co['post_root_id'].astype('int64')
    filtered_df = syn_co[syn_co['pre_root_id'].isin(id_list) & syn_co['post_root_id'].isin(id_list)]
    combinations = filtered_df[['pre_root_id', 'post_root_id']].drop_duplicates().values.tolist()

    for combo in combinations:
            x_temp, y_temp, z_temp = [], [], []
            
            # df1 = neurons[combo[0]]["c"][neurons[combo[0]]["c"]["partner_id"] == combo[1]]
            # df2 = neurons[combo[1]]["n"].connectors[neurons[combo[1]]["n"].connectors["partner_id"] == combo[0]]
            df1 = filtered_df[(filtered_df['pre_root_id'] == combo[0]) & (filtered_df['post_root_id'] == combo[1])]

            # if df1.empty and df2.empty:
            if df1.empty:
                continue
    
            # connectors_df = pd.concat([df1, df2])

            x_temp.extend(df1.loc[:, "x"])
            y_temp.extend(df1.loc[:, "y"])
            z_temp.extend(df1.loc[:, "z"])

            c_coordinates = np.vstack((x_temp, y_temp, z_temp)).T

            idloc1, dist1 = snap_coordinates(neurons[combo[0]]["src"],neurons[combo[0]]["dst"], 
                                             neurons[combo[0]]["s_x"],neurons[combo[0]]["s_y"], neurons[combo[0]]["s_z"],
                                             neurons[combo[0]]["s_x_2"],neurons[combo[0]]["s_y_2"], neurons[combo[0]]["s_z_2"],
                                             neurons[combo[0]]["s_x_3"],neurons[combo[0]]["s_y_3"], neurons[combo[0]]["s_z_3"],
                                             neurons[combo[0]]["d_x"],neurons[combo[0]]["d_y"],neurons[combo[0]]["d_z"],
                                             neurons[combo[0]]["d_x_2"],neurons[combo[0]]["d_y_2"],neurons[combo[0]]["d_z_2"],
                                             neurons[combo[0]]["d_x_3"],neurons[combo[0]]["d_y_3"],neurons[combo[0]]["d_z_3"],
                                            c_coordinates)
            idloc2, dist2 = snap_coordinates(neurons[combo[1]]["src"],neurons[combo[1]]["dst"], 
                                             neurons[combo[1]]["s_x"],neurons[combo[1]]["s_y"], neurons[combo[1]]["s_z"],
                                             neurons[combo[1]]["s_x_2"],neurons[combo[1]]["s_y_2"], neurons[combo[1]]["s_z_2"],
                                             neurons[combo[1]]["s_x_3"],neurons[combo[1]]["s_y_3"], neurons[combo[1]]["s_z_3"],
                                             neurons[combo[1]]["d_x"],neurons[combo[1]]["d_y"],neurons[combo[1]]["d_z"],
                                             neurons[combo[1]]["d_x_2"],neurons[combo[1]]["d_y_2"],neurons[combo[1]]["d_z_2"],
                                             neurons[combo[1]]["d_x_3"],neurons[combo[1]]["d_y_3"],neurons[combo[1]]["d_z_3"],
                                            c_coordinates)
            
            synaptic_s_bef_temp = np.vectorize(neurons[combo[0]]["bef_mapping"].get)(idloc1)
            synaptic_s_af_temp = np.vectorize(neurons[combo[0]]["af_mapping"].get)(idloc1)
            synaptic_d_bef_temp = np.vectorize(neurons[combo[1]]["bef_mapping"].get)(idloc2)
            synaptic_d_af_temp = np.vectorize(neurons[combo[1]]["af_mapping"].get)(idloc2)

            synaptic_src.extend(idloc1)
            synaptic_s_bef.extend(synaptic_s_bef_temp)
            synaptic_s_af.extend(synaptic_s_af_temp)
            synaptic_dst.extend(idloc2)
            synaptic_d_bef.extend(synaptic_d_bef_temp)
            synaptic_d_af.extend(synaptic_d_af_temp)
            n_combo.extend([(f"{combo[0]}_{combo[1]}")] * len(idloc1))
            x.extend(x_temp)
            y.extend(y_temp)
            z.extend(z_temp)
            dists.extend(dist1 + dist2)
            
    connection_type_s = ["s"] * len(synaptic_src)

    return synaptic_src, synaptic_dst, connection_type_s, x, y, z, n_combo, dists, synaptic_s_bef, synaptic_s_af, synaptic_d_bef, synaptic_d_af

In [10]:
def spatial_connectome_creation(edges, n_ds):
    random_number = random.randint(1000, 10000)
    ### create new Graph ### 
    first_elements = edges[:, 0]
    second_elements = edges[:, 1]
    neighbor_matrix = (first_elements[:, np.newaxis] == second_elements[np.newaxis, :])

    # Step 3: Extract neighbor pairs
    neighbor_indices = np.argwhere(neighbor_matrix)
    neighbor_indices = neighbor_indices[neighbor_indices[:, 0] != neighbor_indices[:, 1]]
    neighbor_pairs = edges[neighbor_indices]

    # Step 4: Combine node indices to create unique identifiers for edges
    src_combined = neighbor_pairs[:, 0, 0] * 10000 + neighbor_pairs[:, 0, 1]
    dst_combined = neighbor_pairs[:, 1, 0] * 10000 + neighbor_pairs[:, 1, 1]
    
    # Convert to lists for Arkouda
    src = src_combined.astype(np.int64).tolist()
    dst = dst_combined.astype(np.int64).tolist()

    ### get coordinates and calculate distances of nodes ###
    s_bef = neighbor_pairs[:, 0, 0].astype(np.int64).tolist()
    s_af = neighbor_pairs[:, 0, 1].astype(np.int64).tolist()

    d_bef = neighbor_pairs[:, 1, 0].astype(np.int64).tolist()
    d_af = neighbor_pairs[:, 1, 1].astype(np.int64).tolist()

    ###mapping###
    bef_mapping = dict(zip(src + dst, s_bef + d_bef))
    af_mapping = dict(zip(src + dst, s_af + d_af))

    # Create DataFrame for s_bef and s_af with their corresponding coordinates
    s_bef_coords = n_ds.nodes.set_index('node_id').loc[s_bef][['x', 'y', 'z']].reset_index()
    s_af_coords = n_ds.nodes.set_index('node_id').loc[s_af][['x', 'y', 'z']].reset_index()

    d_bef_coords = n_ds.nodes.set_index('node_id').loc[d_bef][['x', 'y', 'z']].reset_index()
    d_af_coords = n_ds.nodes.set_index('node_id').loc[d_af][['x', 'y', 'z']].reset_index()

    # Ensure lengths match by repeating coordinates for each occurrence
    s_bef_coords_repeated = s_bef_coords.loc[s_bef_coords.index.repeat(s_bef_coords.index.value_counts().reindex(s_bef_coords.index).fillna(1))]
    s_af_coords_repeated = s_af_coords.loc[s_af_coords.index.repeat(s_af_coords.index.value_counts().reindex(s_af_coords.index).fillna(1))]

    d_bef_coords_repeated = d_bef_coords.loc[d_bef_coords.index.repeat(d_bef_coords.index.value_counts().reindex(d_bef_coords.index).fillna(1))]
    d_af_coords_repeated = d_af_coords.loc[d_af_coords.index.repeat(d_af_coords.index.value_counts().reindex(d_af_coords.index).fillna(1))]

    # Extract coordinates as numpy arrays
    x_bef, y_bef, z_bef = s_bef_coords_repeated['x'].values, s_bef_coords_repeated['y'].values, s_bef_coords_repeated['z'].values
    x_af, y_af, z_af = s_af_coords_repeated['x'].values, s_af_coords_repeated['y'].values, s_af_coords_repeated['z'].values

    s_bef_x = x_bef.astype(np.int64).tolist()
    s_bef_y = y_bef.astype(np.int64).tolist()
    s_bef_z = z_bef.astype(np.int64).tolist()
    
    s_af_x = x_af.astype(np.int64).tolist()
    s_af_y = y_af.astype(np.int64).tolist()
    s_af_z = z_af.astype(np.int64).tolist()
    
    s_distances = np.sqrt((x_bef - x_af)**2 + (y_bef - y_af)**2 + (z_bef - z_af)**2).tolist()
    s_x = ((x_bef + x_af)/2).tolist()
    s_y = ((y_bef + y_af)/2).tolist()
    s_z = ((z_bef + z_af)/2).tolist()

    s_x_2 = (0.75*x_bef + 0.25*x_af).tolist()
    s_y_2 = (0.75*y_bef + 0.25*y_af).tolist()
    s_z_2 = (0.75*z_bef + 0.25*z_af).tolist()

    s_x_3 = (0.25*x_bef + 0.75*x_af).tolist()
    s_y_3 = (0.25*y_bef + 0.75*y_af).tolist()
    s_z_3 = (0.25*z_bef + 0.75*z_af).tolist()

    x_bef, y_bef, z_bef = d_bef_coords_repeated['x'].values, d_bef_coords_repeated['y'].values, d_bef_coords_repeated['z'].values
    x_af, y_af, z_af = d_af_coords_repeated['x'].values, d_af_coords_repeated['y'].values, d_af_coords_repeated['z'].values

    d_bef_x = x_bef.astype(np.int64).tolist()
    d_bef_y = y_bef.astype(np.int64).tolist()
    d_bef_z = z_bef.astype(np.int64).tolist()

    d_af_x = x_af.astype(np.int64).tolist()
    d_af_y = y_af.astype(np.int64).tolist()
    d_af_z = z_af.astype(np.int64).tolist()

    # Calculate distances using vectorized operations
    d_distances = np.sqrt((x_bef - x_af)**2 + (y_bef - y_af)**2 + (z_bef - z_af)**2).tolist()
    d_x = ((x_bef + x_af)/2).tolist()
    d_y = ((y_bef + y_af)/2).tolist()
    d_z = ((z_bef + z_af)/2).tolist()

    d_x_2 = (0.75*x_bef + 0.25*x_af).tolist()
    d_y_2 = (0.75*y_bef + 0.25*y_af).tolist()
    d_z_2 = (0.75*z_bef + 0.25*z_af).tolist()

    d_x_3 = (0.25*x_bef + 0.75*x_af).tolist()
    d_y_3 = (0.25*y_bef + 0.75*y_af).tolist()
    d_z_3 = (0.25*z_bef + 0.75*z_af).tolist()

    connection_type_n = ["n"] * len(src)
    
    return src, dst, s_bef, s_bef_x, s_bef_y, s_bef_z, s_af, s_af_x, s_af_y, s_af_z, s_x, s_y, s_z, s_x_2, s_y_2, s_z_2, s_x_3, s_y_3, s_z_3, s_distances, d_bef, d_bef_x, d_bef_y, d_bef_z, d_af, d_af_x, d_af_y, d_af_z, d_x, d_y, d_z, d_x_2, d_y_2, d_z_2, d_x_3, d_y_3, d_z_3, d_distances, connection_type_n, bef_mapping, af_mapping



In [11]:
def overalll(id_list):
    #ensure no duplicate ids
    id_list= list(set(id_list))

    #to remove neurons that failed loading
    id_list_copy= id_list.copy()

    #all of the columns in the new df
    neurons={}
    src, dst, s_bef, s_bef_x, s_bef_y, s_bef_z, s_af, s_af_x, s_af_y, s_af_z, s_x, s_y, s_z, s_x_2, s_y_2, s_z_2, s_x_3, s_y_3, s_z_3, s_distances, d_bef, d_bef_x, d_bef_y, d_bef_z, d_af, d_af_x, d_af_y, d_af_z, d_x, d_y, d_z, d_x_2, d_y_2, d_z_2, d_x_3, d_y_3, d_z_3, d_distances, connection_type_n, n_id = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

    bef_mapping, af_mapping = {}, {}
    i=0
    for id in id_list:
        #load neuron and its connectors
        n= get_neuron_local(id, 3000, 10000)
        # n, c = get_neuron_local(id, 3000, 1000)

        #if loading failed remove from copied list
        if n == None:
             id_list_copy.remove(id)
             continue
        
        #store neuron and connectors information in neuron dict
        neurons[id] = {"n": n}
        # neurons[id] = {"n": n, "c": c}
        edges = n.edges

        ##angst das namen doppelt vergeben werden, fueg noch neuronen id zu namen hinzu
        src_temp, dst_temp, s_bef_temp, s_bef_x_temp, s_bef_y_temp, s_bef_z_temp, s_af_temp, s_af_x_temp, s_af_y_temp, s_af_z_temp, s_x_temp, s_y_temp, s_z_temp, s_x_2_temp, s_y_2_temp, s_z_2_temp, s_x_3_temp, s_y_3_temp, s_z_3_temp, s_distances_temp, d_bef_temp, d_bef_x_temp, d_bef_y_temp, d_bef_z_temp, d_af_temp, d_af_x_temp, d_af_y_temp, d_af_z_temp, d_x_temp, d_y_temp, d_z_temp, d_x_2_temp, d_y_2_temp, d_z_2_temp, d_x_3_temp, d_y_3_temp, d_z_3_temp, d_distances_temp, connection_type_n_temp, bef_mapping, af_mapping = spatial_connectome_creation(edges, n)
        
        neurons[id].update({"src": src_temp, "dst": dst_temp, "s_x": s_x_temp, "s_y": s_y_temp, "s_z": s_z_temp, "s_x": s_x_temp, "s_y": s_y_temp, "s_z": s_z_temp, "s_x_2": s_x_2_temp, "s_y_2": s_y_2_temp, "s_z_2": s_z_2_temp,"s_x_3": s_x_3_temp, "s_y_3": s_y_3_temp, "s_z_3": s_z_3_temp, "d_x": d_x_temp, "d_y": d_y_temp, "d_z": d_z_temp, "d_x_2": d_x_2_temp, "d_y_2": d_y_2_temp, "d_z_2": d_z_2_temp, "d_x_3": d_x_3_temp, "d_y_3": d_y_3_temp, "d_z_3": d_z_3_temp, "bef_mapping": bef_mapping, "af_mapping": af_mapping})
        # print((src_temp))
        src.extend(src_temp)
        dst.extend(dst_temp)
        s_bef.extend(s_bef_temp)
        s_bef_x.extend(s_bef_x_temp)
        s_bef_y.extend(s_bef_y_temp)
        s_bef_z.extend(s_bef_z_temp)
        s_af.extend(s_af_temp)
        s_af_x.extend(s_af_x_temp)
        s_af_y.extend(s_af_y_temp)
        s_af_z.extend(s_af_z_temp)
        s_x.extend(s_x_temp)
        s_y.extend(s_y_temp)
        s_z.extend(s_z_temp)
        s_distances.extend(s_distances_temp)
        d_bef.extend(d_bef_temp)
        d_bef_x.extend(d_bef_x_temp)
        d_bef_y.extend(d_bef_y_temp)
        d_bef_z.extend(d_bef_z_temp)
        d_af.extend(d_af_temp)
        d_af_x.extend(d_af_x_temp)
        d_af_y.extend(d_af_y_temp)
        d_af_z.extend(d_af_z_temp)
        d_x.extend(d_x_temp)
        d_y.extend(d_y_temp)
        d_z.extend(d_z_temp)
        d_distances.extend(d_distances_temp)
        connection_type_n.extend(connection_type_n_temp)
        n_id.extend([id]* len(src_temp))
        print(i)
        i+=1
    
    if id_list_copy == []:
        return None
    
    synaptic_src, synaptic_dst, connection_type_s, x, y, z, n_combo, dists, synaptic_s_bef, synaptic_s_af, synaptic_d_bef, synaptic_d_af= synaptic_data(neurons, id_list_copy)

    spatial_connectome_edge_dict = {
    "src": src + synaptic_src,
    "dst": dst + synaptic_dst,
    "s_bef": s_bef + synaptic_s_bef,
    "s_bef_x": s_bef_x + [0]* len(synaptic_src),
    "s_bef_y": s_bef_y + [0]* len(synaptic_src),
    "s_bef_z": s_bef_z + [0]* len(synaptic_src),
    "s_af": s_af+ synaptic_s_af,
    "s_af_x": s_af_x+ [0]* len(synaptic_src),
    "s_af_y": s_af_y+ [0]* len(synaptic_src),
    "s_af_z": s_af_z+ [0]* len(synaptic_src),
    "s_x": s_x+ x,
    "s_y": s_y+ y,
    "s_z": s_z+ z,
    "s_distance": s_distances+ dists,
    "d_bef": d_bef+ synaptic_d_bef,
    "d_bef_x": d_bef_x+ [0]* len(synaptic_src),
    "d_bef_y": d_bef_y+ [0]* len(synaptic_src),
    "d_bef_z": d_bef_z+ [0]* len(synaptic_src),
    "d_af": d_af+ synaptic_d_af,
    "d_af_x": d_af_x+ [0]* len(synaptic_src),
    "d_af_y": d_af_y+ [0]* len(synaptic_src),
    "d_af_z": d_af_z+ [0]* len(synaptic_src),
    "d_x": d_x+ x,
    "d_y": d_y+ y,
    "d_z": d_z+ z,
    "d_distance": d_distances+ dists,
    "n_id": n_id + n_combo,
    "connection_type": connection_type_n + connection_type_s
    }
    
    # Step 5: Connect to Arkouda and initialize the graph
    ak.connect()
    graph = ar.PropGraph()
    spatial_connectome_edge_df = ak.DataFrame(spatial_connectome_edge_dict)
    graph.load_edge_attributes(spatial_connectome_edge_df, source_column="src", destination_column="dst", 
                               relationship_columns=["s_bef", "s_bef_x", "s_bef_y", "s_bef_z", "s_af", "s_af_x", 
                                                     "s_af_y", "s_af_z", 's_x', "s_y", "s_z", "s_distance", "d_bef",
                                                     "d_bef_x", "d_bef_y", "d_bef_z", "d_af", "d_af_x", "d_af_y", "d_af_z",
                                                       "d_x", "d_y", "d_z", "d_distance", "n_id", "connection_type"])
    ak.disconnect()

    # return graph ,spatial_connectome_edge_df, neurons
    return spatial_connectome_edge_dict

In [12]:
d = overalll(root_ids[:500])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

connected to arkouda server tcp://*:5555
disconnected from arkouda server tcp://*:5555


In [13]:
df = pd.DataFrame(d)
df.to_csv("/home/michaelshewarega/Desktop/test/demo_output.csv", index=False)
# ak.connect()
# d_ak = ak.DataFrame(d)

In [59]:
w = Widget(arkouda_df=d_ak)
w

Widget()

In [15]:
w.current_mapping

{'1': 43233446, '10002': 34463438, '30004': 37222860, '40005': 28602856}

In [16]:
motif_json = [{'label': [0, 1],
  'properties': ['neuron connection', '#000000'],
  'index': 0,
  'indices': [0, 1],
  'tree': None},
 {'label': [1, 2],
  'properties': ['neuron connection', '#000000'],
  'index': 1,
  'indices': [1, 2],
  'tree': None},
 {'label': [3, 4],
  'properties': ['neuron connection', '#00880A'],
  'index': 2,
  'indices': [3, 4],
  'tree': None},
 {'label': [4, 5],
  'properties': ['neuron connection', '#00880A'],
  'index': 3,
  'indices': [4, 5],
  'tree': None},
 {'label': [6, 7],
  'properties': ['synaptic connection', [0, 1], [3, 4]],
  'index': 4,
  'indices': [6, 7],
  'tree': None}]

In [17]:
current_mapping1 = {'1': 43233446, '10002': 34463438, '30004': 37222860, '40005': 28602856}
current_mapping2 = {'1': 59826192, '10002': 61926355, '30004': 52924544, '40005': 45443376}
current_mapping3 = {'1': 61366743, '10002': 67438614, '30004': 12183442, '40005': 34425569}

In [61]:
viewer = Skelescope()
viewer.visualize_motif(d_ak, w.motif_json, w.current_mapping)
viewer

connected to arkouda server tcp://*:5555
disconnected from arkouda server tcp://*:5555


Skelescope(axis_local_primary_points=[0, 0, 0, 0, 1, 0], segments={0: {'parent_segment': -1, 'children_segment…

In [2]:
# df = pd.DataFrame(d)
# df.to_csv("/home/michaelshewarega/Desktop/test/final_output.csv", index=False)
df_downloaded = pd.read_csv("/home/michaelshewarega/Desktop/test/final_output.csv")
# df_downloaded_ak = ak.DataFrame(df_downloaded.to_dict(orient='list'))
df_downloaded

/tmp/ipykernel_638180/2070561768.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_downloaded = pd.read_csv("/home/michaelshewarega/Desktop/test/final_output.csv")


,src,dst,s_bef,s_bef_x,s_bef_y,s_bef_z,s_af,s_af_x,s_af_y,s_af_z,...,d_af,d_af_x,d_af_y,d_af_z,d_x,d_y,d_z,d_distance,n_id,connection_type
0,280121,70028,28,738022,325844,117401,121,746473,329771,108835,...,28,738022,325844,117401,737538.1875,325047.000,117041.421875,1999.659302,720575940627005443,n
1,280121,220028,28,738022,325844,117401,121,746473,329771,108835,...,28,738022,325844,117401,737917.0000,326053.875,117641.187500,671.531616,720575940627005443,n
2,1210141,280121,121,746473,329771,108835,141,748561,330100,108720,...,121,746473,329771,108835,742248.1250,327807.875,113118.312500,12657.466797,720575940627005443,n
3,1210141,1200121,121,746473,329771,108835,141,748561,330100,108720,...,121,746473,329771,108835,746802.1250,329754.375,109198.750000,979.928955,720575940627005443,n
4,1410165,1210141,141,748561,330100,108720,165,749695,330126,107343,...,141,748561,330100,108720,747517.5625,329936.125,108777.859375,2116.711426,720575940627005443,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10289882,110049,3030458,11,0,0,0,49,0,0,0,...,458,0,0,0,699034.0000,250214.000,175520.000000,1571.915429,720575940661316481_720575940641428365,s
10289883,13231302,21952518,1323,0,0,0,1302,0,0,0,...,2518,0,0,0,789760.0000,276970.000,146500.000000,2044.114298,720575940661316481_720575940641428365,s
10289884,13231302,21952518,1323,0,0,0,1302,0,0,0,...,2518,0,0,0,790272.0000,276228.000,146540.000000,1204.049226,720575940661316481_720575940641428365,s
10289885,23922400,36063391,2392,0,0,0,2400,0,0,0,...,3391,0,0,0,802620.0000,275132.000,137020.000000,2394.249760,720575940661316481_720575940641428365,s
